In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN1001_y"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import pickle 

import sys
sys.path.append('../../src')

In [3]:
import models.CNN_model as CNN_model
import training
import evaluation.nn_eval as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../../data/synthetic2/' + 'processed3.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

In [5]:
train_data.shape, validation_data.shape, test_data.shape, train_gt.shape, validation_gt.shape, test_gt.shape, index_of_changes.shape

((900, 8192, 12),
 (100, 8192, 12),
 (1000, 8192, 12),
 (900, 8192, 1),
 (100, 8192, 1),
 (1000, 8192, 1),
 (1000, 4))

## Model

In [6]:
# Model parameters
n_variables = 12
learning_rate = 0.001
batch_size = 10

convolution_levels = 5    #number of convolution to have in the network
pooling_stride = [2, 2, 2, 2, 2]
weights_shape = [[5, n_variables, 32],
                 [5, 32, 64],
                 [5, 64, 64],
                 [5, 64, 64],
                 [5, 64, 64]]
bias_shape = [32, 64, 64, 64, 64]
activation = tf.nn.relu
n_classes = 1

##positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

#variables_weights = np.ones((n_classes,))

In [7]:
print pos_weight

4.442850052


In [8]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [9]:
summary_path = '../../summary_synthetic/train_abrupt_test_gradual/' + exp_name 
checkpoint_path = '../../model_synthetic/train_abrupt_test_gradual/' + exp_name + '/' + exp_name 

In [10]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [11]:
model.build_conv_net(pos_weight_values= pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation,
                     trainable_conv = True,
                     trainable_last = True,)

In [12]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = validation_data,
                    ground_truth_validation = validation_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0)

In [13]:
trainer.train(max_iter = 10000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Training: 0
AUC = 0.203 vs. 0.188
F1 = 0.329 vs. 0.310
Precision = 0.201 vs. 0.185
Recall = 0.908 vs. 0.946
Validation: 0
AUC = 0.179 vs. 0.175
F1 = 0.311 vs. 0.300
Precision = 0.201 vs. 0.177
Recall = 0.686 vs. 1.000
Minibatch Loss= 1.144616
Training: 100
AUC = 0.461 vs. 0.160
F1 = 0.447 vs. 0.271
Precision = 0.350 vs. 0.157
Recall = 0.619 vs. 0.990
Validation: 100
AUC = 0.416 vs. 0.179
F1 = 0.410 vs. 0.300
Precision = 0.341 vs. 0.177
Recall = 0.513 vs. 0.999
Minibatch Loss= 1.003614
Training: 200
AUC = 0.341 vs. 0.145
F1 = 0.421 vs. 0.266
Precision = 0.304 vs. 0.153
Recall = 0.686 vs. 1.000
Validation: 200
AUC = 0.361 vs. 0.176
F1 = 0.398 vs. 0.300
Precision = 0.306 vs. 0.177
Recall = 0.568 vs. 1.000
Minibatch Loss= 0.948375
Training: 300
AUC = 0.628 vs. 0.216
F1 = 0.582 vs. 0.363
Precision = 0.599 vs. 0.222
Recall = 0.565 vs. 0.995
Validation: 300
AUC = 0.579 vs. 0.175
F1 = 0.549 vs. 0.300
Precision = 0.594 vs. 0.177
Recall = 0.510 vs. 1.000
Minibatch Loss= 1.038859
Training: 400
AU

In [14]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../../model_synthetic/train_abrupt_test_gradual/CNN1001_y/CNN1001_y-10000'

In [ ]:
#load the model
#saver = tf.train.Saver()
#saver.restore(sess, "../models/model" + exp_name + "/model")

In [16]:
#This is the model evaluation on test dataset
_= nn_eval.evaluate_model (model, sess, test_data, test_gt)

AUC = 0.784 vs. 0.176
F1 = 0.735 vs. 0.300
Precision = 0.783 vs. 0.177
Recall = 0.692 vs. 1.000


In [17]:
out = nn_eval.deploy(model, sess, test_data, test_gt)

In [18]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
out = np.squeeze(out, axis = 2)

In [19]:
out.shape

(1000, 256)

In [20]:
savemat('../../poc_synthetic/train_abrupt_test_gradual/poc'+exp_name+'.mat', {'poc':out})